In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os.path
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

from pyrejection.experiments import run_experiments
from pyrejection.evaluation import (get_ct_summaries, get_nl_summaries,
                                    get_conditional_error, standard_fig_style,
                                    render_svg_fig)

In [ ]:
metric = 'accuracy'
classifier = 'logreg'
random_state = 0
base_cache_dir='results_cache'

datasets = [
    'exp-noisy-letter-recognition',
]
coverage_configurations = {
    'base': {'nl_rate_pairs': [(1, 0)]},
    'reduced_rate': {'nl_rate_pairs': [(0.4, 0)]},
}

In [ ]:
config_results = {}
for label, config in coverage_configurations.items():
    print('Starting experiments for config:', label)
    cache_dir = os.path.join(base_cache_dir, label)
    config_results[label] = run_experiments(
        [metric],
        [classifier],
        datasets,
        random_states=[random_state],
        worker_count=3,
        cache_dir=cache_dir,
        drop_test_preds=True,
        discard_results=False,
        **config
    )

In [ ]:
tradeoff_rows = []
for label, exp_results in config_results.items():
    for exp_result in exp_results:
        dataset = exp_result['config']['dataset']
        ct_summaries = get_ct_summaries(exp_result)
        for summary in get_ct_summaries(exp_result):
            tradeoff_rows.append({
                'coverage_config': 'ct',
                'dataset': dataset,
                'rejection': 1 - summary['coverage'],
                'conditional_error': get_conditional_error(summary),
            })
        for summary in get_nl_summaries(exp_result):
            tradeoff_rows.append({
                'coverage_config': label,
                'dataset': dataset,
                'rejection': 1 - summary['coverage'],
                'conditional_error': get_conditional_error(summary),
            })
tradeoffs_df = pd.DataFrame(tradeoff_rows)
# Filter fully-rejecting classifiers.
tradeoffs_df = tradeoffs_df[tradeoffs_df['rejection'] < 1]

config_label_map = {
    'ct': 'CT',
    'base': 'NL θ=1',
    'reduced_rate': 'NL θ=0.4  ',
}

def plot_tradeoff(dataset, configs):
    plot_tradeoffs_df = tradeoffs_df.copy()
    plot_tradeoffs_df = plot_tradeoffs_df[(plot_tradeoffs_df['dataset'] == dataset) &
                                          plot_tradeoffs_df['coverage_config'].isin(configs)]
    plot_tradeoffs_df['coverage_config'] = plot_tradeoffs_df['coverage_config'].map(config_label_map)
    colours = sns.color_palette("YlOrRd_r", len(configs)).as_hex()
    fig = px.scatter(plot_tradeoffs_df, x='rejection', y='conditional_error', color='coverage_config',
                     color_discrete_map={config_label_map[key]: colour
                                         for key, colour in zip(configs, colours)})
    standard_fig_style(fig)
    fig.update_layout({
        'width': 550,
        'legend_orientation': 'h',
        'legend': {'x': 0, 'y': 1.1},
        'legend_title_text': '',
        'xaxis_title': 'Rejection Rate',
        'yaxis_title': 'Conditional Error Rate',
        'xaxis': {'range': (-0.05, 1)}
    })
    fig.update_traces({
        'marker': {
            'size': 10,
        }
    })
    print(dataset)
    render_svg_fig(fig)

In [ ]:
plot_tradeoff('exp-noisy-letter-recognition', ['base', 'reduced_rate', 'ct'])